In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import lxml

In [2]:
max_seasons = 34
season_page_url_base = 'https://simpsons.fandom.com/wiki/Season_'
season_page_urls = [season_page_url_base+str(i+1) for i in range(max_seasons)]

In [111]:
def get_links_by_season(season_id: int, verbose=False):
    req = requests.get(season_page_urls[season_id])
    if verbose:
        print(f"Request terminated with status code {req.status_code}")
        print(f"Response encoded with {req.encoding}")
    season_page = BeautifulSoup(req.text, 'html.parser')

    episodes_links = {"title" : [], "url": [], "episode": [], "season": []}

    tables = season_page('table')
    rows = tables[4].find_all('tr')
    episode_num = (len(rows)-1)
    for i in range(episode_num-1):
        col = rows[i].find_all('td')
        if len(col) == 8:
            link = col[2].find_all('a', href=True)
            episodes_links['title'].append(link[0].text)
            episodes_links['url'].append('https://simpsons.fandom.com'+link[0]['href'])
            episodes_links['season'].append(season_id+1)
            episodes_links['episode'].append(i//2+1)

    return episodes_links

In [114]:
get_links_by_season(0)
links_dict = [get_links_by_season(i) for i in range(max_seasons)]
links_df = pd.DataFrame(links_dict[0])
links_df.head(10)

,title,url,episode,season
0,Simpsons Roasting on an Open Fire,https://simpsons.fandom.com/wiki/Simpsons_Roas...,1,1
1,Bart the Genius,https://simpsons.fandom.com/wiki/Bart_the_Genius,2,1
2,Homer's Odyssey,https://simpsons.fandom.com/wiki/Homer%27s_Ody...,3,1
3,There's No Disgrace Like Home,https://simpsons.fandom.com/wiki/There%27s_No_...,4,1
4,Bart the General,https://simpsons.fandom.com/wiki/Bart_the_General,5,1
5,Moaning Lisa,https://simpsons.fandom.com/wiki/Moaning_Lisa,6,1
6,The Call of the Simpsons,https://simpsons.fandom.com/wiki/The_Call_of_t...,7,1
7,The Telltale Head,https://simpsons.fandom.com/wiki/The_Telltale_...,8,1
8,Life on the Fast Lane,https://simpsons.fandom.com/wiki/Life_on_the_F...,9,1
9,Homer's Night Out,https://simpsons.fandom.com/wiki/Homer%27s_Nig...,10,1


In [93]:
req = requests.get('https://www.imdb.com/title/tt0096697/episodes?season=1')
print(f"Request terminated with status code {req.status_code}")
print(f"Response encoded with {req.encoding}")

Request terminated with status code 200
Response encoded with UTF-8


In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
#this time we use the "requests" library from PyPI, which has the typical GET and POST functions
url = "https://en.wikipedia.org/wiki/List_of_FIFA_World_Cup_finals"
req = requests.get(url)
print(f"Request terminated with status code {req.status_code}")
print(f"Response encoded with {req.encoding}")
# as before, we can add the HTML to our soup
fifa_soup = BeautifulSoup(req.text, 'html.parser')

Request terminated with status code 200
Response encoded with UTF-8
